In [1]:
%load_ext autoreload
%autoreload 2

import particle as pt
import uproot
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import binned_statistic
from scipy.optimize import curve_fit
import matplotlib.mlab as mlab
import time
from matplotlib.colors import LogNorm
import uproot.models
from pprint import pprint
from tqdm import tqdm
import polars as pl
import sys
import os
import particle
# using getlogin() returning username
user_name = os.getlogin()

sys.path.append(f'/home/{user_name}/Documents/Atmos_Pandora/apc_atmo_repo/Anatree/')
sys.path.append(f'/home/henrique/Documents/Atmos_Pandora/apc_atmo_repo/personal/Henrique/Analysis/pida/')
from anatree_class import Anatree
from PIDA_class import PIDA
from  ana_tools import *
import plotly.graph_objects as go


import mplhep
mplhep.style.use(mplhep.style.ROOT)
plt.rcParams.update({'font.size': 23,
                     'grid.linestyle': '--',
                     'axes.grid': True,
                     'figure.autolayout': True,
                     'figure.figsize': [14,6],
                     })


from pida_functions import *

In [2]:
def loaddata(path:str, entry_stop=None, forceit=False):
    parquet_path= path.replace(".root", ".parquet")
    if os.path.isfile(parquet_path) and not forceit:
        print(path, 'loaded parquet')
        df = pl.read_parquet(parquet_path)
        return df
    pid = PIDA(path, entry_stop=entry_stop)
    dfs:pl.DataFrame
    dfv:pl.DataFrame
    df:pl.DataFrame

    dfs = pid.dfsingles
    dfv = pid.dfvector
    dfp = pid.dfplanes

    df = dfs.join(dfv, on=selection_events())
    df = df.join(dfp, on=selection_events())
    df.write_parquet(file=parquet_path)
    return df
    
# df = loaddata("data/pida_new_full.root", entry_stop=None, forceit=False)
df = pl.scan_parquet("data/pida_new_full.parquet")
# df = pl.scan_parquet('data/pida_new.parquet')
# df = pl.scan_parquet('data/pida_new.parquet')


In [3]:
dftrkg4 = getbestof(df)

In [4]:
dfall = dftrkg4.sort('trklen').filter(
    pl.col('ccnc_truth') == 0,
    pl.col('nuPDG_truth').abs() == 12
)
dfall = dfall.with_columns(
    pnc = pl.col('trkpurity_planes_B')*pl.col('trkcompleteness_planes_B'),
)


In [5]:
dftmp:pl.DataFrame
if isinstance(dfall, pl.LazyFrame):
    dftmp = dfall.group_by(selection_events(), maintain_order=True).agg(
    ).collect()
    # dftmp = dftmp.head(int(len(dftmp)*0.2))
    # dftmp = dftmp
    # dfall = dfall.join(dftmp.lazy(), on=selection_events()).collect()
    if isinstance(dfall, pl.LazyFrame):
        dfall = dfall.collect()

In [7]:
dfall['trkshwlen']

trkshwlen
f32
-999.0
-999.0
-999.0
-999.0
-999.0
…
557.832764
328.443115
325.856964
